In [3]:
import pandas as pd 
import numpy as np
import pycatch22 as tsfe

In [ ]:
df_options = pd.read_csv('InitialOptionsData.csv')

In [ ]:
df_options.tail()

,secid,date,exdate,cp_flag,strike_price,volume,open_interest,impl_volatility,opprc,moneyness,tte,close,spread,noi
72548453,213987,20191231,20200221,P,25000,1,1,0.760760,1.90,0.910747,52,27.45,0.8,0
72548454,213987,20191231,20200221,P,30000,4,1,0.695719,4.40,1.092896,52,27.45,0.4,0
72548455,213987,20191231,20200221,P,35000,0,1,0.689794,8.25,1.275046,52,27.45,0.9,0
72548456,213987,20191231,20200515,P,22500,0,13,0.824336,2.80,0.819672,136,27.45,0.8,0
72548457,213987,20191231,20200515,P,25000,0,1,0.808585,3.90,0.910747,136,27.45,1.2,0


In [ ]:
np.savetxt(r'secid.txt', df_options.secid.unique(), fmt='%d')

In [ ]:
df_crsp_connector = pd.read_csv('secid_to_crsp.csv')

In [ ]:
df_crsp_connector.head()

,secid,sdate,edate,PERMNO,score
0,7621,19960101,20211231,78091,1
1,8170,19960101,20211231,80341,1
2,100861,19960102,20211231,76868,1
3,100862,19960102,20211231,54594,1
4,100863,19960102,20211231,80498,1


In [ ]:
np.savetxt(r'permno.txt', df_crsp_connector.PERMNO.unique(), fmt='%d')

In [ ]:
df_tickers = pd.read_csv('permno_to_tickers.csv')
df_tickers.head()

,TICKER,PERMNO,NCUSIP,sdate,edate,SCORE
0,0001,14392.0,26878510,20140220,20190522.0,1
1,0001,NaN,NaN,20190620,NaN,6
2,000R,14378.0,14163310,20140220,20200210.0,1
3,000V,14423.0,15117E10,20140320,20160322.0,1
4,000V,14423.0,28249U10,20160414,20220331.0,1


In [ ]:
df_tickers['TICKER'].to_csv('tickers.csv', index=False)

In [ ]:
df_estimates = pd.read_csv('InitialEstimatesData.csv')

In [ ]:
df_estimates.describe().to_csv('InitalExploration.csv')

In [ ]:
df_estimates.head()

,OFTIC,TICKER,CUSIP,CNAME,STATPERS,MEASURE,FISCALP,ESTFLAG,NUMEST,NUMUP,NUMDOWN,MEDEST,MEANEST,STDEV,HIGHEST,LOWEST,FPEDATS,ACTUAL,ANNDATS_ACT
0,PE,0045,70187710,PARSLEY EGY,20180118,EPS,LTG,P,3,1,0,40.00,68.40,62.91,140.50,24.7,NaN,NaN,NaN
1,PE,0045,70187710,PARSLEY EGY,20180215,EPS,LTG,P,4,1,1,101.72,98.45,79.51,170.36,20.0,NaN,NaN,NaN
2,PE,0045,70187710,PARSLEY EGY,20180315,EPS,LTG,P,4,1,2,58.81,76.52,65.45,170.36,18.1,NaN,NaN,NaN
3,PE,0045,70187710,PARSLEY EGY,20180419,EPS,LTG,P,4,1,0,60.44,77.33,65.21,170.36,18.1,NaN,NaN,NaN
4,PE,0045,70187710,PARSLEY EGY,20180517,EPS,LTG,P,3,3,1,58.12,52.57,20.76,70.00,29.6,NaN,NaN,NaN


In [ ]:
df_estimates.CNAME.unique()

array(['PARSLEY EGY', 'ORION ENG CARB', 'ENERGOUS', ..., 'ZUMIEZ',
       'ZUORA', 'ZYNERBA PHARMS'], dtype=object)

In [ ]:
'APPLE' in df_estimates.CNAME.unique()

True

In [ ]:
df_estimates.set_index

# Get Earnings Number to Homogenize Dates across Tickers

In [4]:
df_actual_est = pd.read_csv('ActualEarningsData.csv').sort_values(['TICKER', 'ANNDATS'], ascending=[True, False]).set_index('TICKER')
df_actual_est.head()

,CUSIP,OFTIC,CNAME,PENDS,MEASURE,PDICITY,ANNDATS,VALUE
TICKER,,,,,,,,
0001,26878510,EPEGQ,EP ENGR CORP,20191231,EPS,QTR,20200325,NaN
000V,28249U10,EIGR,EIGER,20220630,EPS,QTR,20220804,-0.51
000V,28249U10,EIGR,EIGER,20220331,EPS,QTR,20220505,-0.64
000V,28249U10,EIGR,EIGER,20211231,EPS,QTR,20220310,-0.64
000V,28249U10,EIGR,EIGER,20210930,EPS,QTR,20211104,-0.65


In [5]:
df_actual_est['ERNUM'] = df_actual_est.groupby(
    'TICKER', 
    group_keys = False
).apply(
    lambda x: (x.ANNDATS != x.ANNDATS.shift(1)).cumsum()[::-1]
)
max_ernum = df_actual_est.ERNUM.max()
df_actual_est['ERNUM'] = df_actual_est.ERNUM.groupby(
    'TICKER', 
    group_keys=False
).apply(
    lambda x: x + (max_ernum - x.max())
)

# Try Homogenizing Dates for Estimates

In [6]:
actual_est = df_actual_est.set_index('ANNDATS', append=True)

In [7]:
df_estimates = pd.read_csv('InitialEstimatesData.csv')
df_estimates = df_estimates[df_estimates.FISCALP == 'QTR']
df_estimates.FPEDATS = df_estimates.FPEDATS.astype('int')
estimates = df_estimates.set_index(['TICKER', 'FPEDATS'])

In [8]:
estimates = estimates.loc[estimates.index.get_level_values(0).isin(actual_est.index.get_level_values(0))].sort_index()
estimates['ERNUM'] = 0

In [9]:
for idx, row in actual_est.iterrows():
    try:
        estimates.loc[
            (
                idx[0], 
                slice(idx[1])
            ),
            'ERNUM'
        ] = row.ERNUM
    except:
        pass

In [10]:
estimates = estimates.droplevel(1).set_index('ERNUM', append=True).sort_index()

In [11]:
estimates

OFTIC     CUSIP           CNAME  STATPERS MEASURE FISCALP  \
TICKER ERNUM                                                             
0045   12       PE  70187710     PARSLEY EGY  20180118     EPS     QTR   
       12       PE  70187710     PARSLEY EGY  20180215     EPS     QTR   
       12       PE  70187710     PARSLEY EGY  20180315     EPS     QTR   
       12       PE  70187710     PARSLEY EGY  20180419     EPS     QTR   
       12       PE  70187710     PARSLEY EGY  20180118     EPS     QTR   
...            ...       ...             ...       ...     ...     ...   
ZYNE   5      ZYNE  98986X10  ZYNERBA PHARMS  20180920     EPS     QTR   
       5      ZYNE  98986X10  ZYNERBA PHARMS  20181018     EPS     QTR   
       5      ZYNE  98986X10  ZYNERBA PHARMS  20181115     EPS     QTR   
       5      ZYNE  98986X10  ZYNERBA PHARMS  20181220     EPS     QTR   
       5      ZYNE  98986X10  ZYNERBA PHARMS  20190117     EPS     QTR   

             ESTFLAG  NUMEST  NUMUP  NUMDOWN  MEDEST  MEANEST  STDEV  HIGHEST  \
TICKER ERNUM                                                                    
0045   12          P      30     13        1    0.18     0.19   0.04     0.26   
       12          P      31      6       18    0.17     0.17   0.03     0.25   
       12          P      29     13       10    0.22     0.22   0.06     0.33   
       12          P      30      9        6    0.24     0.23   0.04     0.31   
       12          P      27     10        0    0.21     0.23   0.07     0.38   
...              ...     ...    ...      ...     ...      ...    ...      ...   
ZYNE   5           P       3      0        0   -0.71    -0.72   0.05    -0.68   
       5           P       2      0        0   -0.74    -0.74   0.04    -0.71   
       5           P       3      1        0   -0.62    -0.63   0.07    -0.56   
       5           P       3      0        0   -0.62    -0.63   0.07    -0.56   
       5           P       3      0        0   -0.62    -0.63   0.07    -0.56   

              LOWEST  ACTUAL  ANNDATS_ACT  
TICKER ERNUM                               
0045   12       0.09    0.30   20180221.0  
       12       0.09    0.30   20180221.0  
       12       0.13    0.31   20180503.0  
       12       0.13    0.31   20180503.0  
       12       0.12    0.31   20180503.0  
...              ...     ...          ...  
ZYNE   5       -0.77   -0.50   20190806.0  
       5       -0.77   -0.50   20190806.0  
       5       -0.70   -0.44   20191106.0  
       5       -0.70   -0.44   20191106.0  
       5       -0.70   -0.44   20191106.0  

[66050 rows x 17 columns]

# Time Series Feature Extraction for AAPL

In [12]:
ret, col = [], 'MEDEST'
for ticker, ernum in estimates.index.unique():
    ftrs = pd.DataFrame(tsfe.catch22_all(estimates.loc[(ticker, ernum), col])).set_index('names').T
    ftrs.columns.name = None
    ftrs['ticker'] = ticker
    ftrs['ernum'] = ernum
    ftrs.set_index(['ticker', 'ernum'], inplace=True)
    ret.append(ftrs)

In [13]:
all_ftrs = pd.concat(ret).add_prefix(col)
all_ftrs.head()

,,MEDESTDN_HistogramMode_5,MEDESTDN_HistogramMode_10,MEDESTCO_f1ecac,MEDESTCO_FirstMin_ac,MEDESTCO_HistogramAMI_even_2_5,MEDESTCO_trev_1_num,MEDESTMD_hrv_classic_pnn40,MEDESTSB_BinaryStats_mean_longstretch1,MEDESTSB_TransitionMatrix_3ac_sumdiagcov,MEDESTPD_PeriodicityWang_th0_01,...,MEDESTFC_LocalSimple_mean1_tauresrat,MEDESTDN_OutlierInclude_p_001_mdrmd,MEDESTDN_OutlierInclude_n_001_mdrmd,MEDESTSP_Summaries_welch_rect_area_5_1,MEDESTSB_BinaryStats_diff_longstretch0,MEDESTSB_MotifThree_quantile_hh,MEDESTSC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1,MEDESTSC_FluctAnal_2_dfa_50_1_2_logi_prop_r1,MEDESTSP_Summaries_welch_rect_centroid,MEDESTFC_LocalSimple_mean3_stderr
ticker,ernum,,,,,,,,,,,,,,,,,,,,,
0045,12,0.442759,0.442759,1.812903,4.0,0.356548,0.077495,0.862745,8.0,0.074074,9.0,...,0.076923,0.615385,-0.769231,0.450123,4.0,1.966248,0.318182,0.500000,0.589049,0.894500
00AI,5,-0.264912,-1.142641,5.798794,7.0,0.324564,0.141152,0.500000,10.0,0.166667,7.0,...,0.052632,0.725490,-0.509804,0.724173,3.0,1.809463,0.285714,0.571429,0.098175,0.624293
00AO,5,-1.237429,0.051727,1.504236,6.0,0.412515,-1.382388,0.549020,10.0,0.008074,10.0,...,0.333333,-0.500000,0.384615,0.321774,3.0,1.785271,0.500000,0.727273,0.589049,1.064959
00Q7,5,0.244146,0.465983,1.416789,3.0,0.362395,-0.367887,0.818182,10.0,0.062500,3.0,...,0.142857,0.411765,-0.794118,0.380338,3.0,2.007420,0.461538,0.538462,0.687223,1.008803
01AF,5,0.794838,0.505333,0.775114,3.0,0.187127,-0.014208,0.784314,7.0,0.005867,6.0,...,0.500000,-0.788462,-0.240385,0.035520,4.0,2.072794,0.636364,0.272727,1.178097,1.254924
